In [17]:
import numpy as np

In [18]:
# Load the data
data = np.loadtxt("databp.csv", delimiter=',', skiprows=0)  # skiprows=0 to not skip the header
print(data)

[[0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 1.]
 [2. 1.]
 [5. 1.]
 [3. 3.]
 [3. 1.]
 [1. 1.]
 [1. 2.]
 [1. 3.]
 [1. 1.]
 [3. 0.]
 [2. 0.]
 [3. 1.]
 [4. 1.]
 [4. 1.]
 [3. 0.]
 [4. 0.]
 [5. 0.]
 [5. 0.]
 [0. 0.]
 [0. 2.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 2.]
 [4. 1.]
 [4. 2.]
 [4. 2.]
 [3. 3.]
 [0. 2.]
 [2. 2.]
 [1. 1.]
 [2. 0.]
 [2. 1.]
 [1. 2.]
 [2. 0.]
 [2. 0.]
 [1. 1.]
 [4. 2.]
 [1. 2.]
 [1. 3.]
 [3. 2.]
 [2. 1.]
 [2. 2.]
 [2. 4.]
 [0. 3.]
 [0. 1.]
 [1. 2.]
 [0. 2.]
 [2. 3.]
 [0. 3.]
 [2. 3.]
 [1. 4.]
 [1. 1.]
 [2. 2.]
 [0. 3.]
 [1. 3.]
 [0. 4.]
 [0. 2.]
 [0. 1.]
 [0. 2.]
 [0. 4.]
 [0. 3.]
 [0. 2.]
 [2. 3.]
 [1. 2.]
 [3. 4.]
 [2. 4.]
 [0. 4.]
 [1. 4.]
 [1. 4.]
 [0. 3.]
 [0. 3.]
 [0. 1.]
 [0. 2.]
 [0. 0.]
 [0. 3.]
 [0. 1.]
 [0. 3.]
 [0. 1.]
 [0. 0.]
 [0. 1.]
 [1. 0.]
 [3. 3.]
 [4. 1.]
 [0. 0.]
 [1. 1.]
 [0. 1.]
 [0. 1.]
 [1. 4.]
 [1. 4.]
 [1. 5.]
 [1. 3.]
 [1. 4.]
 [1. 3.]
 [1. 2.]
 [0. 3.]
 [2. 4.]
 [1. 2.]
 [1. 0.]
 [2. 1.]
 [2. 0.]
 [2. 1.]
 [1. 3.]
 [0. 3.]
 [0. 2.]
 [0. 3.]
 [0. 3.]
 [0. 4.]
 

In [19]:
# Center the data
def center_data(data):
    means = data.mean(axis=0)
    Y = data - means
    return Y

In [20]:
centered_data = center_data(data)
print(centered_data)

[[-1.324 -1.546]
 [-1.324 -1.546]
 [-0.324 -0.546]
 [-0.324 -0.546]
 [ 0.676 -0.546]
 [ 3.676 -0.546]
 [ 1.676  1.454]
 [ 1.676 -0.546]
 [-0.324 -0.546]
 [-0.324  0.454]
 [-0.324  1.454]
 [-0.324 -0.546]
 [ 1.676 -1.546]
 [ 0.676 -1.546]
 [ 1.676 -0.546]
 [ 2.676 -0.546]
 [ 2.676 -0.546]
 [ 1.676 -1.546]
 [ 2.676 -1.546]
 [ 3.676 -1.546]
 [ 3.676 -1.546]
 [-1.324 -1.546]
 [-1.324  0.454]
 [-1.324 -1.546]
 [-1.324 -1.546]
 [-1.324 -1.546]
 [-0.324  0.454]
 [ 2.676 -0.546]
 [ 2.676  0.454]
 [ 2.676  0.454]
 [ 1.676  1.454]
 [-1.324  0.454]
 [ 0.676  0.454]
 [-0.324 -0.546]
 [ 0.676 -1.546]
 [ 0.676 -0.546]
 [-0.324  0.454]
 [ 0.676 -1.546]
 [ 0.676 -1.546]
 [-0.324 -0.546]
 [ 2.676  0.454]
 [-0.324  0.454]
 [-0.324  1.454]
 [ 1.676  0.454]
 [ 0.676 -0.546]
 [ 0.676  0.454]
 [ 0.676  2.454]
 [-1.324  1.454]
 [-1.324 -0.546]
 [-0.324  0.454]
 [-1.324  0.454]
 [ 0.676  1.454]
 [-1.324  1.454]
 [ 0.676  1.454]
 [-0.324  2.454]
 [-0.324 -0.546]
 [ 0.676  0.454]
 [-1.324  1.454]
 [-0.324  1.45

In [21]:
# Fit VAR model
def fit_VAR(data, p):
    n, k = data.shape
    X = center_data(data)
    
    # Construct the regressor matrix Xprime
    Xprime = np.zeros((n - p, k * p))
    X_target = np.zeros((n - p, k))
    
    for t in range(p, n):
        Xprime_row = []
        for lag in range(1, p + 1):
            Xprime_row.extend(X[t - lag])
        Xprime[t - p, :] = Xprime_row
        X_target[t - p] = X[t]
    
    # Estimate coefficients using OLS
    beta = np.linalg.inv(Xprime.T @ Xprime) @ Xprime.T @ X_target
    # Predicting X_hat
    X_hat = Xprime @ beta
    residuals = X[p:] - X_hat
    
    return residuals, beta

In [22]:
residuals, beta = fit_VAR(data, 3)
print(beta)

# changement de forme pour adapter et comparer a l'output de la fonction de fit statsmodels
beta_reshaped = beta.T.reshape(2, 3, 2)
print(beta_reshaped)

[[ 0.54582268 -0.0363546 ]
 [-0.03676121  0.34523054]
 [ 0.07441853  0.03822122]
 [-0.08199956  0.10992764]
 [ 0.00463139 -0.03343979]
 [ 0.06659193  0.07957331]]
[[[ 0.54582268 -0.03676121]
  [ 0.07441853 -0.08199956]
  [ 0.00463139  0.06659193]]

 [[-0.0363546   0.34523054]
  [ 0.03822122  0.10992764]
  [-0.03343979  0.07957331]]]


In [23]:
# Function to estimate VAR-ARMA parameters
def estimate_VAR_ARMA(data, p, residuals): #p = ordre du VAR sur les donnees centrees, necessaire pour les retards et valeurs manquantes des residus
    T, N = data.shape
    assert N == 2, "This implementation is for bivariate VAR only."
    
    n = T - p  # Number of rows after adjusting for lags
    Xprime = np.zeros((n, 5)) #n lignes pour les pbs, 3 = 1 cst + 2 termes retard X_t-1 + 2 terme retard et-1
    Xprime[:, 0] = 1  # colonne de la constante 
    
    for t in range(0, n):
        Xprime[t, 1:3] = data[t + p , :] #pour t= 0 le terme le plus lointain dans nos données (celui de retard 1) est le (p+1)eme element de nos données, et le premier qui a un terme de retard correspondant
        Xprime[t, 3:5] = residuals[t, :]
    
    X1 = data[p:, 0]
    X2 = data[p:, 1]
    
    beta = np.zeros((Xprime.shape[1], 2))  # Adjusting the dimensions of beta
    
    # Using pseudo-inverse instead of inverse
    beta[:, 0] = np.linalg.pinv(Xprime.T @ Xprime) @ Xprime.T @ X1
    beta[:, 1] = np.linalg.pinv(Xprime.T @ Xprime) @ Xprime.T @ X2
    
    A = np.zeros((2, 2))
    B = np.zeros((2, 2))
    omega = [beta[0, 0], beta[0, 1]]
    A[0, :] = -beta[3, :]
    A[1, :] = -beta[4, :]
    B[0, :] = beta[1, :] - beta[3, :]
    B[1, :] = beta[2, :] - beta[4, :]
    
    return omega, A, B

In [24]:
# Assuming `data` and `residuals` are already defined
p = 1
omega, A, B = estimate_VAR_ARMA(data, p, residuals)
print("Omega:\n", omega)
print("A:\n", A)
print("B:\n", B)

Omega:
 [0.6580226213856607, 0.3614870058120859]
A:
 [[ 0.09887836  0.06243287]
 [-0.09032261  0.41555608]]
B:
 [[ 0.74795296  0.08147566]
 [-0.21329086  1.16634334]]
